# **Introduction to Deep Computer Vision**

In [1]:
import sys
import tensorflow as tf

print("Python is running from:", sys.executable)
print("TensorFlow version:", tf.__version__)

Python is running from: c:\Users\jinny\.conda\envs\gpu\python.exe
TensorFlow version: 2.10.1


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import threading

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image

In [4]:
# Mount the files from Google Drive

#**Image Classification using Tensorflow Kreas**

## **Prepare Flower Dataset**

In [3]:
# Set image size and other parameters
IMAGE_SIZE = 50
input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)  # Adjust based on your dataset image size
target_size = (IMAGE_SIZE, IMAGE_SIZE)
batch_size = 4
num_classes = 5  # 5 flower categories: daisy, dandelion, rose, sunflower, tulip

# Image Data Augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,          # Normalize pixel values
    validation_split = 0.1
    # rotation_range=20,          # Randomly rotate images
    # width_shift_range=0.2,      # Horizontal shift
    # height_shift_range=0.2,     # Vertical shift
    # shear_range=0.2,            # Shear transformation
    # zoom_range=0.2,             # Random zoom
    # horizontal_flip=True,       # Flip images horizontally
    # fill_mode='nearest'         # Fill missing pixels after transformation
)

path_to_train_data = r'C:\Users\jinny\Desktop\cv\flower dataset\train'

# Create the training generator(90%)
train_generator = train_datagen.flow_from_directory(
    path_to_train_data,       # Set the path to your training dataset (5 folders)
    target_size= target_size,     # Resize images to match input_shape
    batch_size=batch_size,
    class_mode='categorical',    # Multiclass classification
    subset = 'training'

)
#Create the validation generator(10%)
valid_generator = train_datagen.flow_from_directory(
#####=====Enter your program code here. =====#####
    path_to_train_data,       
    target_size= target_size,    
    batch_size=batch_size,
    class_mode='categorical',    
    subset = 'validation'

)

# Image Data Augmentation for validation data (only rescaling)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load validation data
path_to_validation_data = r'C:\Users\jinny\Desktop\cv\flower dataset\train'

#Create the test generator

test_generator = test_datagen.flow_from_directory(
#####=====Enter your program code here. =====#####
    path_to_validation_data,
    target_size=target_size,
    batch_size=batch_size,
    class_mode= None

)


# Calculate validation steps
validation_steps = valid_generator.samples // valid_generator.batch_size


Found 880 images belonging to 5 classes.
Found 96 images belonging to 5 classes.
Found 976 images belonging to 5 classes.


In [4]:
batch_train = train_generator.__next__()
x_batch, y_batch = batch_train
print(f"x train {x_batch.shape}, y train {y_batch.shape}")

x train (4, 50, 50, 3), y train (4, 5)


In [5]:
batch_test  = valid_generator.__next__()
x_batch, y_test = batch_test
print(f"x test: {x_batch.shape}, label {y_test}")

x test: (4, 50, 50, 3), label [[0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]


## **Create your CNNs model** </font>
#Convolution -> ReLU -> Pooling -> Flatten -> Dense -> Output ->


In [8]:
# Enter the code as follows:
# Build the Deep Convolutional Neural Network

# Step1: Create sequential stack of layers into a model
model = Sequential()

# Step2: First Convolutional Block
model.add(Conv2D(32, (3,3), activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
# Setp3 Second Convolutional Block
model.add(Conv2D(64, (3,3),
                 activation='relu',
                 padding='same'
                 
                 ))
model.add(MaxPooling2D(pool_size=(2,2)))

# Step4: Third Convolutional Block
model.add(Conv2D(128, (3,3),
                 activation = 'relu',
                 padding = 'same'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Step5: Flatten and Fully Connected Layer
model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.5))

# Step6: Define the Output Layer
model.add(Dense(num_classes, activation='softmax'))
# Step7: Compile the Model
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy']

              )

# EarlyStopping based on accuracy reaching 80%
early_stop_accuracy = EarlyStopping(
    monitor='accuracy',        # Monitor accuracy during training
    baseline=0.01,             # Target accuracy (90%)
    patience=5,                # Stop if no improvement after reaching 90% accuracy
    restore_best_weights=True  # Restore the model weights at the highest accuracy
)

model.summary()

# Step8: Train the model with EarlyStopping
model.fit(
    train_generator,
    epochs = 10,
    validation_data = valid_generator,
    validation_steps = validation_steps,
    callbacks = [early_stop_accuracy]
)

# Step9: Save the model
model.save('flower_model.h5')



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 6, 6, 128)       

In [9]:
# Evaluate the model on validation data
loss, accuracy = model.evaluate(valid_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

24/24 [==============================] - 0s 9ms/step - loss: 0.9079 - accuracy: 0.6771
Validation Accuracy: 67.71%


# **Classify unseen data**

In [10]:
 #####===== Enter your code program. =====#####
# Load the trained model

# Function to preprocess the input image
def preprocess_image(img_path): # For unseen data

    # Step1: Load images for directory path and resize image to match the input shape of the model
    img_array = image.load_img(
        img_path,
        target_size = target_size
    )

    # Step2: Convert the image to a NumPy array
    img_array = image.array_to_img(img_array)
    # Step3: Add a batch dimension (for a single image)
    img_array = np.expand_dims(img_array, axis=0)

    # Step4: Normalize the image (same normalization used during training)
    img_array = img_array/255.0
    # Step5: return image
    return img_array

# Path to the image you want to classify
img_path = r'C:\Users\jinny\Desktop\cv\flower dataset\test\Image_6.jpg'

# Preprocess the image
pre_data = preprocess_image(img_path)

# Use the model to predict the class
predicted_class = model.predict(pre_data)

# Get the index of the highest probability class
predicted_class = np.argmax(predicted_class)

# Class labels (corresponding to the order of your folder structure during training)
class_labels = ['Daisy', 'Dandelion', 'Rose', 'Sunflower', 'Tulip']

# Print the predicted class
print(f"Predicted class: {class_labels[predicted_class]}")

1/1 [==============================] - 0s 409ms/step
Predicted class: Tulip
